![cyber_photo](cyber_photo.jpg)

Cyber threats are a growing concern for organizations worldwide. These threats take many forms, including malware, phishing, and denial-of-service (DOS) attacks, compromising sensitive information and disrupting operations. The increasing sophistication and frequency of these attacks make it imperative for organizations to adopt advanced security measures. Traditional threat detection methods often fall short due to their inability to adapt to new and evolving threats. This is where deep learning models come into play.

Deep learning models can analyze vast amounts of data and identify patterns that may not be immediately obvious to human analysts. By leveraging these models, organizations can proactively detect and mitigate cyber threats, safeguarding their sensitive information and ensuring operational continuity.

As a cybersecurity analyst, you identify and mitigate these threats. In this project, you will design and implement a deep learning model to detect cyber threats. The BETH dataset simulates real-world logs, providing a rich source of information for training and testing your model. The data has already undergone preprocessing, and we have a target label, `sus_label`, indicating whether an event is malicious (1) or benign (0).

By successfully developing this model, you will contribute to enhancing cybersecurity measures and protecting organizations from potentially devastating cyber attacks.


### The Data

| Column     | Description              |
|------------|--------------------------|
|`processId`|The unique identifier for the process that generated the event - int64 |
|`threadId`|ID for the thread spawning the log - int64|
|`parentProcessId`|Label for the process spawning this log - int64|
|`userId`|ID of user spawning the log|Numerical - int64|
|`mountNamespace`|Mounting restrictions the process log works within - int64|
|`argsNum`|Number of arguments passed to the event - int64|
|`returnValue`|Value returned from the event log (usually 0) - int64|
|`sus_label`|Binary label as suspicous event (1 is suspicious, 0 is not) - int64|

More information on the dataset: [BETH dataset](accreditation.md)

In [37]:
# Make sure to run this cell to use torchmetrics. If you cannot use pip install to install the torchmetrics, you can use sklearn.
!pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable


In [38]:
# Import required libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as functional
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torchmetrics import Accuracy
# from sklearn.metrics import accuracy_score  # uncomment to use sklearn

In [39]:
# Load preprocessed data
train_df = pd.read_csv('labelled_train.csv')
test_df = pd.read_csv('labelled_test.csv')
val_df = pd.read_csv('labelled_validation.csv')

# View the first 5 rows of training set
train_df.head()

,processId,threadId,parentProcessId,userId,mountNamespace,argsNum,returnValue,sus_label
0,381,7337,1,100,4026532231,5,0,1
1,381,7337,1,100,4026532231,1,0,1
2,381,7337,1,100,4026532231,0,0,1
3,7347,7347,7341,0,4026531840,2,-2,1
4,7347,7347,7341,0,4026531840,4,0,1


In [40]:
# Start coding here
# Use as many cells as you need
X_train = train_df.drop(columns='sus_label', axis=1).values
y_train = train_df['sus_label'].values
# X_train.head()

In [41]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
# X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns)
# X_train.head()

In [42]:
X_test = test_df.drop(columns='sus_label', axis=1).values
y_test = test_df['sus_label'].values
X_val = val_df.drop(columns='sus_label', axis=1).values
y_val = val_df['sus_label'].values

In [43]:
X_test = scaler.transform(X_test)


X_val = scaler.transform(X_val)


In [44]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1,1)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1,1)

In [45]:
X = torch.tensor(X_train_scaled, dtype=torch.float32)
y = torch.tensor(y_train, dtype=torch.float32).view(-1,1)
print(X.shape)
print(y.shape)
X_train_scaled.shape

torch.Size([763144, 7])
torch.Size([763144, 1])


(763144, 7)

In [46]:
y

tensor([[1.],
        [1.],
        [1.],
        ...,
        [0.],
        [0.],
        [0.]])

In [47]:
dataset = TensorDataset(X, y)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [48]:
model = nn.Sequential(
        nn.Linear(X.shape[1], 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64,1),
        nn.Sigmoid()
)
model

Sequential(
  (0): Linear(in_features=7, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=1, bias=True)
  (5): Sigmoid()
)

In [49]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=model.parameters(), lr=0.001, weight_decay=0.0001)

In [50]:
epochs = 10
for epoch in range(epochs):
    
    model.train()
    epoch_loss = 0.0
    
    for batch_features, batch_labels in dataloader:
        
        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    
    print(f"Epoch: {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")
    
print("Training Complete")
        

    

In [51]:
model.eval()

with torch.no_grad():
    y_predict_train = model(X).round()
    y_predict_test = model(X_test_tensor).round()
    y_predict_val = model(X_val_tensor).round()
    
    

In [52]:
accuracy = Accuracy(task="binary")

train_accuracy = accuracy(y_predict_train, y)
test_accuracy = accuracy(y_predict_test, y_test_tensor)
val_accuracy = accuracy(y_predict_val, y_val_tensor)

print("Training accuracy: {0}".format(train_accuracy))
print("Validation accuracy: {0}".format(val_accuracy))
print("Testing accuracy: {0}".format(test_accuracy))

Training accuracy: 0.7992659211158752
Validation accuracy: 0.9130483269691467
Testing accuracy: 0.08159096539020538


In [53]:
train_accuracy = train_accuracy.item()
test_accuracy = test_accuracy.item()
val_accuracy = val_accuracy.item()

print("Training accuracy: {}".format(train_accuracy))
print("Test accuracy: {}".format(test_accuracy))
print("Val Accurcy: {}".format(test_accuracy))

Training accuracy: 0.7992659211158752
Test accuracy: 0.08159096539020538
Val Accurcy: 0.08159096539020538
